In [ ]:
from pathlib import Path
from genetic_algorithm import genetic_algorithm
from genetic_algorithm.selection import TournamentSelection
from mwcnf_parser import parse_mwcnf
from solver import MwcnfGenerator
from solver.fitness import satisfied_clause_count, weights
from solver.mwcnf_individual import MwcnfIndividual
from weighted_formula.weighted_formula import WeightedCnf

from io import StringIO
import matplotlib.pyplot as plt


In [ ]:
def print_plot(debug_stream: StringIO,
               title: str,
               out_file: Path | None = None):
    orders = []
    bests = []
    means = []
    medians = []
    worsts = []
    for line in debug_stream.getvalue().splitlines():
        order, best, mean_, median_, worst = line.split(sep=',')
        orders.append(abs(int(order)))
        bests.append(abs(float(best)))
        means.append(abs(float(mean_)))
        medians.append(abs(float(median_)))
        worsts.append(abs(float(worst)))

    plt.plot(orders, bests, orders, means, orders, medians, orders, worsts)
    plt.title(title)

    if out_file is not None:
        plt.savefig(out_file)

    plt.close()
    # plt.show()

In [ ]:
def evaluate_hyperparameters(formula_file: str | Path,
                             population_size: int,
                             generation_cnt: int,
                             tournament_size: float,
                             crossover_probability: float,
                             mutation_rate: float,
                             elitism: int,
                             do_print_plot: bool = True):
    debug_stream = StringIO()

    formula_path = Path(formula_file)
    formula = parse_mwcnf(formula_path)
    solution, i, _ = genetic_algorithm(MwcnfGenerator(formula),
                                    population_size=population_size,
                                    number_of_generations=generation_cnt,
                                    selection=TournamentSelection(tournament_size),
                                    crossover_probability=crossover_probability,
                                    mutation_rate=mutation_rate,
                                    elitism=elitism,
                                    terminate_on_stagnation_in_x_generations=300,
                                    debug_stream=debug_stream if do_print_plot else None)

    if do_print_plot:
        print_plot(debug_stream, str(formula_file))

    if isinstance(solution, MwcnfIndividual):
        print(f"Number of generations: {i}")
        print(f"Number satisfied: {-satisfied_clause_count(formula, solution.config)}")
        print(f"{formula_path.name} {-weights(formula, solution.config)} {solution.config.get_evaluation()} 0")

evaluate_hyperparameters("wuf100-430/wuf100-430-Q/wuf100-01.mwcnf",
                         population_size=50,
                         generation_cnt=300,
                         tournament_size=1.75,
                         crossover_probability=0.9,
                         mutation_rate=0.01,
                         elitism=1,
                         do_print_plot=True)

In [ ]:
from itertools import product


instances = [
    # "wuf100-430/wuf100-430-Q/wuf100-01.mwcnf",
    # "wuf75-325/wuf75-325-Q/wuf75-01.mwcnf",
    # "wuf50-218/wuf50-218-Q/wuf50-01.mwcnf",
    # "wuf36-157/wruf36-157-Q/wruf36-157-1.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-01.mwcnf",
]
population_sizes = [
    # 350,
    # 200,
    100,
    50,
]
generation_counts = [
    # 1000,
    # 350,
    200,
    100,
    50,
]
tournament_sizes = [
    # 2.5,
    # 1.9,
    1.75,
    1.5,
    1.25,
    1.1,
]
crossovers = [
    0.9,
    0.5,
    0.1,
]
mutation_rates = [
    0.02,
    0.015,
    0.01,
    0.005,
]
stagnations = [
    None,
]

combinations = list(product(instances, population_sizes, generation_counts, tournament_sizes, crossovers, mutation_rates, stagnations))
len(combinations)

In [ ]:
from datetime import datetime
raise Exception("You've run that, idiot.")
out_folder = Path(f"out{str(datetime.now())}")
out_folder.mkdir()

stats_file = Path(out_folder, "stats.csv")
for instance, tour_size, cross, mut in combinations:
    for i in range(10):
        debug_stream = StringIO()
        formula_path = Path(instance)
        formula = parse_mwcnf(formula_path)
        solution, actual_generation_cnt, _ = genetic_algorithm(MwcnfGenerator(formula),
                                                            population_size=pop_size,
                                                            number_of_generations=gen_cnt,
                                                            selection=TournamentSelection(tour_size),
                                                            crossover_probability=cross,
                                                            mutation_rate=mut,
                                                            elitism=1,
                                                            debug_stream=debug_stream)
        print_plot(debug_stream,
                title=f"{formula_path.name} {i} {pop_size=} {gen_cnt=} {tour_size=} {cross=} {mut=}",
                out_file=Path(out_folder, f"{formula_path.name}-{i}-{pop_size}-{gen_cnt}-{tour_size}-{cross}-{mut}.png"))
        with open(Path(out_folder, f"{formula_path.name}-{i}-{pop_size}-{gen_cnt}-{tour_size}-{cross}-{mut}.csv"), mode="wt+") as stats_f:
            stats_f.write(debug_stream.getvalue())

        with open(stats_file, mode="at+") as stats_f:
            clause_cnt = formula.clause_cnt
            sat_clause_cnt = -satisfied_clause_count(formula, solution.config)
            weight = -weights(formula, solution.config)
            stats_f.write(f"{formula_path.name},{i},{clause_cnt},{sat_clause_cnt},{weight},{pop_size},{gen_cnt},{tour_size},{cross},{mut}\n")


In [ ]:
from dataclasses import dataclass
from math import ceil, floor


actual_max_weights = {
    "wruf36-157-1.mwcnf": 23667,
    "wruf36-157-2.mwcnf": 18540,
    "wruf36-157-3.mwcnf": 12400,
    "wruf36-157-4.mwcnf": 17728,
    "wruf36-157-5.mwcnf": 14944,
    "wruf36-157-6.mwcnf": 9883,
    "wruf36-157-7.mwcnf": 3984,
    "wruf36-157-8.mwcnf": 13591,
    "wruf36-157-9.mwcnf": 7031,

}


@dataclass
class Result:
    formula_name: str
    run_no: int
    clause_cnt: int
    sat_clause_cnt: int
    weight: int
    max_weight: int
    pop_size: int
    gen_cnt: int
    tour_size: float
    cross: float
    mut: float
    last_improvement: int

    def solved(self):
        return self.clause_cnt == self.sat_clause_cnt

    def get_int(self):
        return self.solved() * self.weight / self.max_weight

    def get_penalized_last_improvement(self):
        return (10 - (self.solved() * 9)) * self.last_improvement

stats = {}
with open("out/stats.csv", "rt") as stats_f:
    for line in stats_f.readlines():
        formula_name, run_no, clause_cnt, sat_clause_cnt, weight, max_weight, pop_size, \
            gen_cnt, tour_size, cross, mut = line.split(sep=",")
        file_name = f"out/{formula_name}-{run_no}-{tour_size}-{cross}-{float(mut)}"
        current_best = 0
        last_improvement = 0
        with open(file_name, mode="rt") as debug_f:
            for line in debug_f.readlines():
                generation, best = line.split(sep=",")[:2]
                if ceil(float(best)) < current_best:
                    last_improvement = int(generation)
                    current_best = ceil(float(best))

        dict_key = f"{tour_size}-{cross}-{float(mut)}"
        stats.setdefault(dict_key, []).append(Result(formula_name,
                                                     int(run_no),
                                                     int(clause_cnt),
                                                     int(sat_clause_cnt),
                                                     int(weight),
                                                     int(actual_max_weights[formula_name]),
                                                     int(pop_size),
                                                     int(gen_cnt),
                                                     float(tour_size),
                                                     float(cross),
                                                     float(mut),
                                                     int(last_improvement)))

# stats

In [ ]:
from copy import deepcopy
from numpy import mean

filtered_stats = deepcopy(stats)
# for key, result_list in stats.items():
#     if result_list[0].mut == 0.02 or result_list[0].tour_size < 1.5 \
#         or result_list[0].pop_size != 100 or result_list[0].gen_cnt != 200:
#         filtered_stats.pop(key)

averages = []
for key, result_list in filtered_stats.items():
    averages.append(
        (mean([result.get_int() for result in result_list]),
         mean([result.get_penalized_last_improvement() for result in result_list]),
         mean([result.last_improvement for result in result_list]),
         key)
    )

averages.sort(reverse=True)
print("\n".join([str(avg) for avg in averages[:10]]))

averages.sort(key=lambda x: x[1])
print("\n".join([str(avg) for avg in averages[:10]]))

# for succ_avg, ppz_avg, key in averages[:10]:
#     results = stats[key]
#     for result in results:
#         file_name = f"out/{result.formula_name}-{result.run_no}-{result.tour_size}-{result.cross}-{float(result.mut)}"
#         debug_stream = StringIO()
#         with open(file_name, "rt") as f:
#             debug_stream.writelines(f.readlines())

#         print_plot(debug_stream=debug_stream,
#                    title=file_name,
#                    out_file=Path(file_name + ".png"))

In [ ]:
selected_hyperparameters = {
    "population_size": 100,
    "number_of_generations": 500,
    "selection": TournamentSelection(1.5),
    "crossover_probability": 0.9,
    "mutation_rate": 0.01,
    "elitism": 1
}

instances = [
    "wuf20-91/wuf20-91-Q/wuf20-01.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-02.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-03.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-04.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-05.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-06.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-07.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-08.mwcnf",
    "wuf20-91/wuf20-91-Q/wuf20-09.mwcnf",
]

solved = 0
for instance in instances:
    for i in range(10):
        debug_stream = StringIO()
        formula_path = Path(instance)
        formula = parse_mwcnf(formula_path)
        solution, actual_generation_cnt, _ = genetic_algorithm(MwcnfGenerator(formula),
                                                            debug_stream=debug_stream,
                                                            **selected_hyperparameters)
        print_plot(debug_stream,
                   title=f"{formula_path.name}")

        clause_cnt = formula.clause_cnt
        sat_clause_cnt = -satisfied_clause_count(formula, solution.config)
        weight = -weights(formula, solution.config)
        print(f"{clause_cnt=}, {sat_clause_cnt=}, {weight=}")
        solved += clause_cnt == sat_clause_cnt

solved